In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 

dta = sm.datasets.fair.load_pandas().data

In [2]:
dta["affair"] = (dta.affairs > 0).astype(int)
y, X = dmatrices("affair ~ rate_marriage + age + yrs_married + children + religious + educ + C(occupation) + C(occupation_husb)", dta, return_type="dataframe")
X = X.rename(columns =
{"C(occupation)[T.2.0]":"occ_2",
"C(occupation)[T.3.0]":"occ_3",
"C(occupation)[T.4.0]":"occ_4",
"C(occupation)[T.5.0]":"occ_5",
"C(occupation)[T.6.0]":"occ_6",
"C(occupation_husb)[T.2.0]":"occ_husb_2",
"C(occupation_husb)[T.3.0]":"occ_husb_3",
"C(occupation_husb)[T.4.0]":"occ_husb_4",
"C(occupation_husb)[T.5.0]":"occ_husb_5",
"C(occupation_husb)[T.6.0]":"occ_husb_6"})
y = np.ravel(y)

In [17]:
X.head()

,occ_2,occ_3,occ_4,occ_5,occ_6,occ_husb_2,occ_husb_3,occ_husb_4,occ_husb_5,occ_husb_6,rate_marriage,age,yrs_married,children,religious,educ
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,32.0,9.0,3.0,3.0,17.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,27.0,13.0,3.0,1.0,14.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,22.0,2.5,0.0,1.0,16.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,37.0,16.5,4.0,3.0,16.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,27.0,9.0,1.0,1.0,14.0


In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6366 entries, 0 to 6365
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Intercept      6366 non-null   float64
 1   occ_2          6366 non-null   float64
 2   occ_3          6366 non-null   float64
 3   occ_4          6366 non-null   float64
 4   occ_5          6366 non-null   float64
 5   occ_6          6366 non-null   float64
 6   occ_husb_2     6366 non-null   float64
 7   occ_husb_3     6366 non-null   float64
 8   occ_husb_4     6366 non-null   float64
 9   occ_husb_5     6366 non-null   float64
 10  occ_husb_6     6366 non-null   float64
 11  rate_marriage  6366 non-null   float64
 12  age            6366 non-null   float64
 13  yrs_married    6366 non-null   float64
 14  children       6366 non-null   float64
 15  religious      6366 non-null   float64
 16  educ           6366 non-null   float64
dtypes: float64(17)
memory usage: 895.2 KB


### Implementing the model

In [4]:
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.542911
         Iterations 6
                          Results: Logit
Model:              Logit            Pseudo R-squared: 0.137      
Dependent Variable: y                AIC:              6946.3465  
Date:               2022-07-04 20:10 BIC:              7061.2449  
No. Observations:   6366             Log-Likelihood:   -3456.2    
Df Model:           16               LL-Null:          -4002.5    
Df Residuals:       6349             LLR p-value:      1.5339e-222
Converged:          1.0000           Scale:            1.0000     
No. Iterations:     6.0000                                        
------------------------------------------------------------------
                   Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
Intercept          2.9708   0.5722   5.1917 0.0000  1.8492  4.0923
occ_2              0.3902   0.4476   0.8719 0.3832 -0

In [5]:
X.corr()

,Intercept,occ_2,occ_3,occ_4,occ_5,occ_6,occ_husb_2,occ_husb_3,occ_husb_4,occ_husb_5,occ_husb_6,rate_marriage,age,yrs_married,children,religious,educ
Intercept,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
occ_2,NaN,1.000000,-0.348075,-0.251243,-0.143237,-0.052128,0.183782,-0.020904,-0.009786,-0.093292,-0.059107,-0.019697,-0.034223,0.004668,0.081182,-0.013129,-0.217719
occ_3,NaN,-0.348075,1.000000,-0.560645,-0.319631,-0.116322,-0.000638,0.090043,0.011248,0.003021,-0.101673,-0.053082,-0.066371,-0.021261,-0.063298,-0.034986,-0.335615
occ_4,NaN,-0.251243,-0.560645,1.000000,-0.230712,-0.083962,-0.083123,-0.043159,0.037341,-0.001946,0.085766,0.068882,0.040982,-0.026816,-0.003235,0.043996,0.477505
occ_5,NaN,-0.143237,-0.319631,-0.230712,1.000000,-0.047868,-0.053426,-0.044053,-0.039932,0.114903,0.006016,-0.002109,0.079533,0.076820,0.033274,0.004260,-0.022121
occ_6,NaN,-0.052128,-0.116322,-0.083962,-0.047868,1.000000,-0.046140,-0.029028,-0.043541,-0.030926,0.218824,0.008878,0.030676,-0.004912,-0.026830,0.011784,0.226920
occ_husb_2,NaN,0.183782,-0.000638,-0.083123,-0.053426,-0.046140,1.000000,-0.146849,-0.347951,-0.316693,-0.153248,-0.038992,-0.057368,-0.033451,0.001190,0.009990,-0.160756
occ_husb_3,NaN,-0.020904,0.090043,-0.043159,-0.044053,-0.029028,-0.146849,1.000000,-0.197588,-0.179838,-0.087024,-0.022514,0.011610,0.008046,-0.005538,0.008170,-0.052723
occ_husb_4,NaN,-0.009786,0.011248,0.037341,-0.039932,-0.043541,-0.347951,-0.197588,1.000000,-0.426115,-0.206198,0.003303,-0.048989,-0.031121,-0.008032,-0.008491,-0.031422
occ_husb_5,NaN,-0.093292,0.003021,-0.001946,0.114903,-0.030926,-0.316693,-0.179838,-0.426115,1.000000,-0.187674,0.003256,0.105525,0.092462,0.053965,-0.000063,0.042540


In [6]:
X.drop("Intercept",axis=1,inplace=True) # as correlation is NaN

In [7]:
X.iloc[44]

occ_2             0.0
occ_3             0.0
occ_4             0.0
occ_5             0.0
occ_6             0.0
occ_husb_2        1.0
occ_husb_3        0.0
occ_husb_4        0.0
occ_husb_5        0.0
occ_husb_6        0.0
rate_marriage     4.0
age              22.0
yrs_married       2.5
children          0.0
religious         1.0
educ             14.0
Name: 44, dtype: float64

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [14]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [15]:
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [20]:
pipe.score(X_test, y_test)

0.7303664921465969

In [21]:
import bz2,pickle
file = bz2.BZ2File('logistic_regression.pkl','wb')
pickle.dump(pipe,file)
file.close()